In [1]:
import pandas as pd

from src.dataset import process_dataset

df = pd.read_csv('../res/dataset.csv')
df

,hosp_0,diag_0,dead_0,hosp_1,diag_1,dead_1,hosp_2,diag_2,dead_2,hosp_3,...,diag_39,dead_39,hosp_40,diag_40,dead_40,hosp_41,diag_41,dead_41,init_zone,actuated_zone
0,98.0,7137.0,420.0,106.0,7255.0,421.0,107.0,7344.0,424.0,116.0,...,10723.0,550.0,143.0,10759.0,552.0,135.0,10778.0,553.0,Y,R
1,224.0,9389.0,463.0,219.0,9524.0,470.0,223.0,9652.0,474.0,227.0,...,11416.0,576.0,78.0,11444.0,577.0,83.0,11525.0,578.0,R,W
2,119.0,10800.0,556.0,109.0,10817.0,558.0,100.0,10840.0,559.0,91.0,...,13564.0,619.0,199.0,13710.0,621.0,196.0,13822.0,627.0,W,Y
3,94.0,11575.0,578.0,96.0,11674.0,580.0,106.0,11774.0,580.0,111.0,...,15288.0,700.0,144.0,15309.0,701.0,138.0,15324.0,706.0,Y,R
4,193.0,13887.0,633.0,198.0,14057.0,635.0,199.0,14202.0,640.0,206.0,...,15779.0,732.0,35.0,15807.0,734.0,36.0,15840.0,735.0,R,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,128.0,10591.0,541.0,121.0,10614.0,543.0,104.0,10628.0,547.0,89.0,...,10847.0,566.0,9.0,10854.0,566.0,10.0,10868.0,566.0,R,O
1396,15.0,10763.0,565.0,14.0,10764.0,565.0,13.0,10765.0,565.0,11.0,...,11016.0,571.0,18.0,11017.0,571.0,13.0,11017.0,572.0,O,R
1397,10.0,10875.0,566.0,13.0,10891.0,566.0,16.0,10903.0,566.0,16.0,...,11220.0,575.0,15.0,11231.0,575.0,14.0,11244.0,576.0,R,W
1398,11.0,11018.0,573.0,6.0,11020.0,574.0,5.0,11022.0,574.0,4.0,...,11729.0,586.0,46.0,11750.0,588.0,36.0,11759.0,593.0,W,O


In [2]:
x, y = process_dataset(df)
x.shape, y.shape

((1400, 53), (1400, 2))